In [1]:
%load_ext autoreload
%autoreload 2


import sys
import warnings


warnings.filterwarnings("ignore")
sys.path.append("../")

## IO markup

### Train

In [2]:
from modules.data import bert_data

In [3]:
train_df_path = "/home/eartemov/ae/work/factRuEval-2016/dev.csv"
valid_df_path = "/home/eartemov/ae/work/factRuEval-2016/test.csv"

In [4]:
data = bert_data.LearnData.create(
    train_df_path=train_df_path,
    valid_df_path=valid_df_path,
    idx2labels_path="/home/eartemov/ae/work/factRuEval-2016/idx2labels.txt",
    clear_cache=True
)

The pre-trained model you are loading is a cased model but you have not set `do_lower_case` to False. We are setting `do_lower_case=False` for you but you may want to check this behavior.


In [5]:
from modules.models.bert_models import BERTBiLSTMAttnCRF

In [6]:
model = BERTBiLSTMAttnCRF.create(len(data.train_ds.idx2label), crf_dropout=0.3)

In [7]:
from modules.train.train import NerLearner

In [8]:
num_epochs = 100

In [9]:
learner = NerLearner(
    model, data, "/home/eartemov/ae/work/models/fre-BERTBiLSTMAttnCRF-IO.cpt", t_total=num_epochs * len(data.train_dl))

In [10]:
model.get_n_trainable_params()

2629497

In [13]:
learner.fit(epochs=num_epochs)

INFO:root:Resuming train... Current epoch 58.


INFO:root:
epoch 59, average train epoch loss=0.00029528



INFO:root:on epoch 45 by max_f1: 0.91


              precision    recall  f1-score   support

         I_O      0.981     0.987     0.984     48875
       I_LOC      0.890     0.834     0.861      1557
       I_PER      0.960     0.968     0.964      2112
       I_ORG      0.863     0.774     0.816      3865

   micro avg      0.971     0.968     0.969     56409
   macro avg      0.923     0.891     0.906     56409
weighted avg      0.970     0.968     0.968     56409



### Eval

#### Load model

In [14]:
from modules.data import bert_data

In [15]:
data = bert_data.LearnData.create(
    train_df_path=train_df_path,
    valid_df_path=valid_df_path,
    idx2labels_path="/home/eartemov/ae/work/factRuEval-2016/idx2labels.txt",
    clear_cache=False
)

INFO:pytorch_pretrained_bert.tokenization:loading vocabulary file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-multilingual-cased-vocab.txt from cache at /home/eartemov/.pytorch_pretrained_bert/96435fa287fbf7e469185f1062386e05a075cadbf6838b74da22bf64b080bc32.99bcd55fc66f4f3360bc49ba472b940b8dcf223ea6a345deb969d607ca900729


In [16]:
from modules.models.bert_models import BERTBiLSTMAttnCRF

In [17]:
model = BERTBiLSTMAttnCRF.create(len(data.train_ds.idx2label), crf_dropout=0.3)

INFO:pytorch_pretrained_bert.modeling:loading archive file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-multilingual-cased.tar.gz from cache at /home/eartemov/.pytorch_pretrained_bert/731c19ddf94e294e00ec1ba9a930c69cc2a0fd489b25d3d691373fae4c0986bd.4e367b0d0155d801930846bb6ed98f8a7c23e0ded37888b29caa37009a40c7b9
INFO:pytorch_pretrained_bert.modeling:extracting archive file /home/eartemov/.pytorch_pretrained_bert/731c19ddf94e294e00ec1ba9a930c69cc2a0fd489b25d3d691373fae4c0986bd.4e367b0d0155d801930846bb6ed98f8a7c23e0ded37888b29caa37009a40c7b9 to temp dir /tmp/tmpa98t5m09
INFO:pytorch_pretrained_bert.modeling:Model config {
  "attention_probs_dropout_prob": 0.1,
  "directionality": "bidi",
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "max_position_embeddings": 512,
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pooler_fc_size": 768,
  "pooler_num_attention_heads": 12,
  "

In [18]:
from modules.train.train import NerLearner

In [19]:
num_epochs = 100

In [20]:
learner = NerLearner(
    model, data, "/home/eartemov/ae/work/models/fre-BERTBiLSTMAttnCRF-IO.cpt", t_total=num_epochs * len(data.train_dl))

In [21]:
model.get_n_trainable_params()

2629497

In [22]:
learner.load_model()

### Predict

In [23]:
from modules.data.bert_data import get_data_loader_for_predict

In [24]:
dl = get_data_loader_for_predict(data, df_path=data.valid_ds.config["df_path"])

In [25]:
preds = learner.predict(dl)

In [26]:
from sklearn_crfsuite.metrics import flat_classification_report

In [27]:
from modules.analyze_utils.utils import bert_labels2tokens, voting_choicer
from modules.analyze_utils.plot_metrics import get_bert_span_report

In [28]:
pred_tokens, pred_labels = bert_labels2tokens(dl, preds)
true_tokens, true_labels = bert_labels2tokens(dl, [x.bert_labels for x in dl.dataset])

In [29]:
assert pred_tokens == true_tokens
tokens_report = flat_classification_report(true_labels, pred_labels, labels=["I_ORG", "I_PER", "I_LOC"], digits=4)

In [30]:
print(tokens_report)

              precision    recall  f1-score   support

       I_ORG     0.8850    0.7622    0.8190      3865
       I_PER     0.9598    0.9725    0.9661      2112
       I_LOC     0.9318    0.8163    0.8702      1557

   micro avg     0.9178    0.8324    0.8730      7534
   macro avg     0.9255    0.8504    0.8851      7534
weighted avg     0.9156    0.8324    0.8708      7534



In [31]:
from modules.analyze_utils.plot_metrics import get_bert_span_report
from modules.analyze_utils.utils import voting_choicer, tokens2spans

In [32]:
print(get_bert_span_report(dl, preds, fn=voting_choicer))

              precision    recall  f1-score   support

         LOC     0.8368    0.7418    0.7864      1313
           O     0.9764    0.9877    0.9820     48753
         ORG     0.7478    0.5954    0.6629      1893
         PER     0.8763    0.8568    0.8664      1306

    accuracy                         0.9644     53265
   macro avg     0.8593    0.7954    0.8244     53265
weighted avg     0.9623    0.9644    0.9630     53265

